In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

import pandas as pd


Define Evaluation Metrics

Since  predicting a continuous variable (block_median_fee_rate), regression metrics like:

Mean Absolute Error (MAE)

Root Mean Squared Error (RMSE)

R² Score

In [4]:
!pip install xgboost statsmodels tensorflow


In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from statsmodels.tsa.arima.model import ARIMA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense



In [6]:
df = pd.read_csv('bitcoin_data_cleaned.csv')
df = df.set_index('block_time')
df = df.drop(columns=['timestamp'])

# Define features (X) and target (y)
X = df.drop(columns=['block_median_fee_rate'])
y = df['block_median_fee_rate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [7]:
# Store performance metrics for each model
results = {}

# 1. Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_y_pred = rf_model.predict(X_test)
results['Random Forest'] = {
    'MAE': mean_absolute_error(y_test, rf_y_pred),
    'RMSE': np.sqrt(mean_squared_error(y_test, rf_y_pred)),
    'R²': r2_score(y_test, rf_y_pred)
}



In [8]:
# 2. XGBoost
xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)
xgb_y_pred = xgb_model.predict(X_test)
results['XGBoost'] = {
    'MAE': mean_absolute_error(y_test, xgb_y_pred),
    'RMSE': np.sqrt(mean_squared_error(y_test, xgb_y_pred)),
    'R²': r2_score(y_test, xgb_y_pred)
}



In [9]:
# 3. LSTM Model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(units=50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Reshape data for LSTM
X_train_lstm = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_lstm = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

lstm_model = create_lstm_model((X_train_lstm.shape[1], 1))
lstm_model.fit(X_train_lstm, y_train, epochs=10, batch_size=32, verbose=2)
lstm_y_pred = lstm_model.predict(X_test_lstm)
results['LSTM'] = {
    'MAE': mean_absolute_error(y_test, lstm_y_pred),
    'RMSE': np.sqrt(mean_squared_error(y_test, lstm_y_pred)),
    'R²': r2_score(y_test, lstm_y_pred)
}



Epoch 1/10


/Users/jiangqinma/miniconda3/envs/bitcoin/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1198/1198 - 7s - 6ms/step - loss: 3.7803e-04
Epoch 2/10
1198/1198 - 6s - 5ms/step - loss: 2.1289e-07
Epoch 3/10
1198/1198 - 6s - 5ms/step - loss: 7.7129e-08
Epoch 4/10
1198/1198 - 6s - 5ms/step - loss: 3.5196e-06
Epoch 5/10
1198/1198 - 6s - 5ms/step - loss: 2.0264e-06
Epoch 6/10
1198/1198 - 7s - 6ms/step - loss: 2.9701e-06
Epoch 7/10
1198/1198 - 6s - 5ms/step - loss: 2.3219e-06
Epoch 8/10
1198/1198 - 6s - 5ms/step - loss: 1.8181e-06
Epoch 9/10
1198/1198 - 6s - 5ms/step - loss: 2.9743e-06
Epoch 10/10
1198/1198 - 6s - 5ms/step - loss: 7.4574e-07
300/300 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [12]:
# 4. Compare results
print("Model Comparison Results:")
for model, metrics in results.items():
    print(f"{model}: MAE = {metrics['MAE']}, RMSE = {metrics['RMSE']}, R² = {metrics['R²']}")


Model Comparison Results:
Random Forest: MAE = 5.991443900256219e-08, RMSE = 1.3488465262316422e-07, R² = 0.9517162245256123
XGBoost: MAE = 3.881243214433048e-07, RMSE = 6.139094598098831e-07, R² = -0.00019527177549383268
LSTM: MAE = 0.0004571340420475348, RMSE = 0.00046049902536819855, R² = -562772.4193305996
